# Simulate proteome time profiles.
## Start from measured proteome that is already hedged: necessary to enable lactate usage later?

In [1]:
import seaborn as sns
import pandas as pd

sns.plt.rcParams['svg.fonttype'] = 'none'
pd.set_option('display.max_colwidth', -1)
#pd.set_option('display.float_format', lambda x: '%.3g' % x)
%matplotlib inline

In [2]:
PROTOTYPE = True

In [3]:
from dynamicme.tests.helpme import HelpME
from dynamicme.dynamic import get_cplx_concs

helper = HelpME(prototyping=PROTOTYPE)
solver = helper.solver
me = solver.me

In [4]:
for data in me.complex_data:
    data._complex_id = data.id    
    form_id0 = 'ES_'+data.id.replace('_E_c','')
    try:
        form = me.reactions.get_by_id(form_id0)
        form.id = 'formation_'+ data.id
    except KeyError:
        form = me.reactions.get_by_id('formation_'+data.id)    

me.repair()

In [5]:
from dynamicme.dynamic import DelayedDynamicME

dyme = DelayedDynamicME(solver)
dyme.delay_model.solver.set_realopts('lp', {'Feasibility tol':1e-15,'Optimality tol':1e-15})
dsolver = dyme.delay_model.solver
dme = dsolver.me

/home/laurence/python_modules/dynamicme.git/dynamicme/dynamic.py:1218 UserWarning: Dynamic mRNA not yet implemented!


In [6]:
dme.metabolites.delayed_abundance_PGI_E_c._bound

2.11395311739584e-5*exp(-0.1*mu)

In [7]:
%%time
sol_max = solver.bisectmu(basis=solver.feas_basis, verbosity=0)

CPU times: user 5.19 s, sys: 6.67 ms, total: 5.19 s
Wall time: 5.15 s


In [8]:
cplx_concs_max = get_cplx_concs(solver)

In [9]:
mu_me = me.solution.x_dict['biomass_dilution']
print 'mu of ME for growth maximization:', mu_me

mu of ME for growth maximization: 0.845703125


# Validate mixed substrate uptake hierarchy--Beg et al. (2007) MSB

In [10]:
import warnings

T = 10   # hours
V = 1.   # L
X0 = 0.00675 / V  # g/L
c0_dict = {'glc__D_e': 0.4,
           'lac__L_e': 0.4,
           'malt_e': 0.4,
           'gal_e': 0.4,
           'glyc_e':0.4,
           'ac_e': 0.0}

### Convert from g/L to mmol
mids_remove = []
for mid,c in c0_dict.iteritems():
    try:        
        met = me.metabolites.get_by_id(mid)    
        c0_dict[met.id] = c / met.formula_weight * 1000
    except KeyError:
        warnings.warn('Model has no metabolite: %s. Will remove it from c0_dict' % mid)
        mids_remove.append(mid)

for mid in mids_remove:
    c0_dict.pop(mid)
        
### Set max uptake rate
LB_EX = -10.
LB_O2 = -20.

lb_dict={}
ub_dict={}

for mid in c0_dict.keys():
    try:
        rxn = dyme.get_exchange_rxn(mid)    
        if rxn.id == 'EX_o2_e':
            lb = LB_O2
        else:
            lb = LB_EX
        rxn.lower_bound = lb
        lb_dict[rxn.id] = lb
    except KeyError:
        warnings.warn('Model has no metabolite: '+ mid)

me.reactions.EX_o2_e.lower_bound = -20.
lb_dict[me.reactions.EX_o2_e.id] = -20.

for k,lb in lb_dict.iteritems():
    rxn = dsolver.me.reactions.get_by_id(k)
    rxn.lower_bound = lb

print c0_dict
print lb_dict

{'ac_e': 0.0, 'glc__D_e': 2.220299442904667}
{'EX_glc_e': -10.0, 'EX_ac_e': -10.0, 'EX_o2_e': -20.0}


/usr/lib/python2.7/site-packages/ipykernel/__main__.py:20 UserWarning: Model has no metabolite: glyc_e. Will remove it from c0_dict
/usr/lib/python2.7/site-packages/ipykernel/__main__.py:20 UserWarning: Model has no metabolite: malt_e. Will remove it from c0_dict
/usr/lib/python2.7/site-packages/ipykernel/__main__.py:20 UserWarning: Model has no metabolite: lac__L_e. Will remove it from c0_dict
/usr/lib/python2.7/site-packages/ipykernel/__main__.py:20 UserWarning: Model has no metabolite: gal_e. Will remove it from c0_dict


## Initialize with the same Exchange bounds as bioreactor simulation

In [11]:
sol0 = solver.bisectmu(basis=solver.feas_basis)

iter	muopt    	a     	b     	mu1       	stat1
Finished substituting S,lb,ub in 0.273165 seconds
Finished makeME_LP in 0.055213 seconds
Getting MINOS parameters from ME_NLP...
1 0.0 0.0 1.0 1.0 1
Finished substituting S,lb,ub in 0.273506 seconds
Finished makeME_LP in 0.054325 seconds
Getting MINOS parameters from ME_NLP...
2 0.5 0.5 1.0 0.5 optimal
Finished substituting S,lb,ub in 0.280806 seconds
Finished makeME_LP in 0.054224 seconds
Getting MINOS parameters from ME_NLP...
3 0.75 0.75 1.0 0.75 optimal
Finished substituting S,lb,ub in 0.277972 seconds
Finished makeME_LP in 0.054083 seconds
Getting MINOS parameters from ME_NLP...
4 0.75 0.75 0.875 0.875 1
Finished substituting S,lb,ub in 0.280069 seconds
Finished makeME_LP in 0.054293 seconds
Getting MINOS parameters from ME_NLP...
5 0.8125 0.8125 0.875 0.8125 optimal
Finished substituting S,lb,ub in 0.281102 seconds
Finished makeME_LP in 0.053804 seconds
Getting MINOS parameters from ME_NLP...
6 0.84375 0.84375 0.875 0.84375 optimal
Fi

In [12]:
cplx_conc_dict0 = get_cplx_concs(solver)

## The static solution should be the same as dynamic solution starting from same initial proteome

In [13]:
dyme.delay_model.update_cplx_concs(cplx_conc_dict0)

In [14]:
sol_dyme = dsolver.bisectmu(basis=dsolver.feas_basis, mumax=10)

iter	muopt    	a     	b     	mu1       	stat1
Finished compiling expressions in 2.742276 seconds
Finished substituting S,lb,ub in 0.316003 seconds
Finished makeME_LP in 0.063061 seconds
Getting MINOS parameters from ME_NLP...
1 5.0 5.0 10 5.0 optimal
Finished substituting S,lb,ub in 0.328479 seconds
Finished makeME_LP in 0.063120 seconds
Getting MINOS parameters from ME_NLP...
2 7.5 7.5 10 7.5 optimal
Finished substituting S,lb,ub in 0.328227 seconds
Finished makeME_LP in 0.062829 seconds
Getting MINOS parameters from ME_NLP...
3 7.5 7.5 8.75 8.75 1
Finished substituting S,lb,ub in 0.309685 seconds
Finished makeME_LP in 0.064297 seconds
Getting MINOS parameters from ME_NLP...
4 7.5 7.5 8.125 8.125 1
Finished substituting S,lb,ub in 0.328244 seconds
Finished makeME_LP in 0.063382 seconds
Getting MINOS parameters from ME_NLP...
5 7.5 7.5 7.8125 7.8125 1
Finished substituting S,lb,ub in 0.327548 seconds
Finished makeME_LP in 0.064139 seconds
Getting MINOS parameters from ME_NLP...
6 7.656

In [15]:
print sol0[0], '=?=', sol_dyme[0]
assert abs(sol_dyme[0] - sol0[0]) < 1e-10

0.845703125 =?= 7.70080566406


AssertionError: 

In [16]:
for k,lb in lb_dict.iteritems():
    rxn0 = solver.me.reactions.get_by_id(k)
    rxnd = dsolver.me.reactions.get_by_id(k)
    print '%-15.10s%-15.10s%-15.10s' % (k, rxn0.x, rxnd.x)

print '%-15.10s%-15.10s%-15.10s' % ('mu', sol0[0], sol_dyme[0])

EX_glc_e       -10.0          -4.0215762     
EX_ac_e        0.54559887     0.24087604     
EX_o2_e        -2.5948627     -0.6589743     
mu             0.84570312     7.70080566     


# DEBUG

In [17]:
dme = dsolver.me

In [18]:
dme.metabolites.delayed_abundance_PGI_E_c._bound

2.26189423649353e-5*exp(-0.1*mu)

In [19]:
for rxn in me.metabolites.ICDHyr_E_c.reactions:
    print '%-15.10s%-35.30s' % (rxn.x, rxn.id)
    #print '%-15.10s%-25.20s%-50s' % (rxn.id, rxn.x, rxn.reaction)

0.29759987     ICDHyr                             
1.07556045     formation_ICDHyr_E_c               


In [20]:
for rxn in dme.metabolites.ICDHyr_E_c.reactions:
    print '%-15.10s%-35.30s' % (rxn.x, rxn.id)

0.0            formation_ICDHyr_E_c               
5.88809963     abundance_ICDHyr_E_c               
-4.5343110     dedt_ICDHyr_E_c                    


In [21]:
dme.reactions.abundance_ICDHyr_E_c.reaction

'mu ICDHyr_E_c + enzyme_capacity_ICDHyr_E_c --> delayed_abundance_ICDHyr_E_c'

In [22]:
met = dme.metabolites.delayed_abundance_ICDHyr_E_c
for rxn in met.reactions:
    print '%-35.30s%-35.30s%-25s' % (met.id, rxn.id, rxn.metabolites[met])

delayed_abundance_ICDHyr_E_c       abundance_ICDHyr_E_c               1                        
delayed_abundance_ICDHyr_E_c       formation_ICDHyr_E_c               -0.1                     


In [23]:
conss = []
for cons in dme.metabolites.query('delayed_abundance'):
    if cons._bound != 0:
        if hasattr(cons._bound, 'subs'):
            conss.append(cons)

In [24]:
print len(conss)

146
